In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile
import pickle

from bidi.algorithm import get_display
import arabic_reshaper

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer
from nltk import wordpunct_tokenize

import PyArabic

In [2]:
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join('.', filename)

    statinfo = os.stat(local_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + local_filename +
                        '. Can you get to it with a browser?')
    return local_filename


filename = maybe_download('webteb.zip',  81750261)

Found and verified webteb.zip


In [3]:
import io
# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    
    with zipfile.ZipFile(filename) as f:
        data = wordpunct_tokenize(tf.compat.as_str(f.read(f.namelist()[0])))
        return data


vocabulary = read_data(filename)
print('Data size', len(set(vocabulary)))

Data size 313999


In [4]:
def preprocess_word(word):
    preprocess = PyArabic.ArabicPreprocessor()
    word = preprocess.deNoise(word)
    return word

def process_data(data):
    
    stopwords_list = stopwords.words('arabic')
    stemmer = ISRIStemmer()
    
    data = [stemmer.stem(word) for word in data if word not in stopwords_list]
    
    return data

In [5]:
#vocabulary = process_data(vocabulary)

In [6]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 10000


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary


# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


Most common words (+UNK) [['UNK', 4131047], ('،', 1418836), ('.', 865446), ('من', 768570), ('في', 732988)]
Sample data [452, 6438, 766, 107, 166, 286, 1678, 867, 6325, 0] ['#', "'", 'فقدان', 'ُ', 'الوزن', 'ِ', 'شعور', 'ٌ', 'رائع', 'UNK']


In [7]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
          '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

6438 ' -> 452 #
6438 ' -> 766 فقدان
766 فقدان -> 107 ُ
766 فقدان -> 6438 '
107 ُ -> 766 فقدان
107 ُ -> 166 الوزن
166 الوزن -> 107 ُ
166 الوزن -> 286 ِ


In [8]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()

with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [10]:
# Step 5: Begin training.
num_steps = 300000
LOG_DIR = './logs/word2vec'

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')
    saver = tf.train.Saver()
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
        
    saver.save(session, os.path.join(LOG_DIR, "model.ckpt"), step)
    final_embeddings = normalized_embeddings.eval()


Initialized
Average loss at step  0 :  239.67185974121094
Nearest to الذي: بذور, وقابلة, مخاطية, قام, واخر, إنتاج, 55, الشحوم,
Nearest to هذه: احتماليه, تلعب, ؟!, الأعمار, ووظائف, مصادر, الأوعية, النباتية,
Nearest to تناول: الشخص, الدهن, إرادة, بحرقة, المستويات, نفعا, الأن, الياسمين,
Nearest to هل: بصل, ابدا, وتصلب, يطول, هضمية, ونمو, للبول, بالحيوان,
Nearest to إن: تكسر, تقييم, حراري, احرصي, مزيد, الشرب, الخط, الجهة,
Nearest to يمكن: خزعة, 00, أصلي, وذمة, أجل, غيرها, معد, إرهاق,
Nearest to مع: قلقة, الصحي, المعقدة, الخطيرة, الهضمية, هده, تجرى, شروق,
Nearest to السؤال: داء, أربع, الصالحة, الزيت, سالبة, قيصرية, تعني, ويسبب,
Nearest to إلى: تغيب, يتأكد, الإشراف, اكتشفت, يمارس, الذكاء, وتقوم, عظمة,
Nearest to لمدة: ازالة, الزيادة, الاصبع, ابدئي, الجنة, لاكثر, ٧, والانتظار,
Nearest to .: علاجية, الدوائين, البروجسترون, لأخرى, كريهه, أخذت, وغازات, المثال,
Nearest to :: الرئوي, بروز, ولايوجد, دور, الذهني, القناة, الساخن, للتوتر,
Nearest to *: والحقيقة, التسمم, تزداد, مسح, النوبة, الاطفال, بقا

In [17]:
with open(os.path.join(LOG_DIR, 'vocabulary.tsv'), 'w') as f:
    [f.write(word + '\n') for word in dictionary.keys()]

In [12]:
os.path.join(LOG_DIR, 'vocabulary.tsv')

'./logs/word2vec/vocabulary.tsv'

In [16]:
from tensorflow.contrib.tensorboard.plugins import projector
# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter(LOG_DIR)

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embeddings.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'vocabulary.tsv')

# Saves a configuration file that TensorBoard will read during startup.
projector.visualize_embeddings(summary_writer, config)

In [ ]:
# Step 6: Visualize the embeddings.


# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        
        # Arabic text preprocessing 
        reshaped_text = arabic_reshaper.reshape(label)
        label = get_display(reshaped_text)
        
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)


try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 400
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, os.path.join('.', 'tsne.png'))

except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)


In [ ]:
pickle.dump(reverse_dictionary, open('vocabulary.pic', 'wb'))
pickle.dump(final_embeddings, open('embeddings.pic', 'wb'))
pickle.dump(dictionary, open('dictionary.pic', 'wb'))

In [ ]:
with open('webteb.d128.txt', 'w', encoding='utf-8') as f:
    for key in reverse_dictionary.keys():
        print(reverse_dictionary[key] + ' ' + ' '.join([str(value) for value in final_embeddings[key]]), file=f)    